In [31]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
import re

In [32]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/sos.csv", encoding = 'CP949')
df.head()

,연,월,일,시,분,수신지역,송출내용,target
0,2020,1,26,9,9,중구,[중구청] 오늘 07:00경 중구 장충동 앰버서더호텔 내 화재 발생. 이 지역을 우...,기타
1,2020,1,28,16,59,광진구,[광진구청] 신종 코로나바이러스감염증 예방수칙 마스크 착용 흐르는물 30초이상 손씻...,전염병
2,2020,1,30,9,49,송파구,[송파구청]감염증 대응을 위해 구청장 중심으로 24시간 대책본부를 운영하고 있습니다...,전염병
3,2020,1,31,14,31,강남구,"[강남구]중국 방문 후 14일 이내에 발열(37.5도 이상) 또는 호흡기증상(기침,...",전염병
4,2020,1,31,15,5,강남구,"[강남구]중국 방문 후 14일이내에 발열(37.5도이상) 또는 호흡기증상(기침,인후...",전염병


In [33]:
# 송출내용, 라벨만 남기고 제거
df = df.drop(columns=['연', '월', '일', '시', '분', '수신지역'])
df.head()

,송출내용,target
0,[중구청] 오늘 07:00경 중구 장충동 앰버서더호텔 내 화재 발생. 이 지역을 우...,기타
1,[광진구청] 신종 코로나바이러스감염증 예방수칙 마스크 착용 흐르는물 30초이상 손씻...,전염병
2,[송파구청]감염증 대응을 위해 구청장 중심으로 24시간 대책본부를 운영하고 있습니다...,전염병
3,"[강남구]중국 방문 후 14일 이내에 발열(37.5도 이상) 또는 호흡기증상(기침,...",전염병
4,"[강남구]중국 방문 후 14일이내에 발열(37.5도이상) 또는 호흡기증상(기침,인후...",전염병


In [34]:
# 라벨 더미변수로 바꾸기
'''
전염병: 0, 태풍: 1, 호우: 2, 한파: 3, 교통: 4, 홍수: 5, 대설:6, 폭염: 7, 산사태: 8, 기타: 9
'''

# 라벨 매핑
label_mapping = {
    '전염병': 0, '태풍': 1, '호우': 2, '한파': 3, '교통': 4,
    '홍수': 5, '대설': 6, '폭염': 7, '산사태': 8, '기타': 9
}


# 라벨을 더미 변수로 변환하여 새로운 데이터프레임 생성
new_df = df.replace({'target': label_mapping})

new_df.head(10)

,송출내용,target
0,[중구청] 오늘 07:00경 중구 장충동 앰버서더호텔 내 화재 발생. 이 지역을 우...,9
1,[광진구청] 신종 코로나바이러스감염증 예방수칙 마스크 착용 흐르는물 30초이상 손씻...,0
2,[송파구청]감염증 대응을 위해 구청장 중심으로 24시간 대책본부를 운영하고 있습니다...,0
3,"[강남구]중국 방문 후 14일 이내에 발열(37.5도 이상) 또는 호흡기증상(기침,...",0
4,"[강남구]중국 방문 후 14일이내에 발열(37.5도이상) 또는 호흡기증상(기침,인후...",0
5,[은평구청]은평성모병원환자코로나19추가확진자발생.성모병원부분일시폐쇄.발열호흡기증상시...,0
6,"[강동구]코로나19 2번째 확진자 발생. 성내동 거주 66세(중국인), 2.24. ...",0
7,"[송파구] 코로나19 확진자 2명 추가 발생, 23일 문정동 확진자의 가족으로 서울...",0
8,"[은평구청]금일코로나19확진추가2명발생,2.1부터은평성모병원입퇴원주민중발열 호흡기증...",0
9,"[강동구] 코로나19 확진자 2명 추가 발생, 자세한내용은 강동구 홈페이지를 참고 ...",0


In [35]:
import re

# 정규 표현식 패턴 설정
pattern = r'\[[^\]]+\]'

# 정규 표현식을 사용하여 패턴을 찾아서 삭제
new_df['송출내용'] = new_df['송출내용'].apply(lambda x: re.sub(pattern, '', x))

new_df.head()

,송출내용,target
0,오늘 07:00경 중구 장충동 앰버서더호텔 내 화재 발생. 이 지역을 우회하여 주...,9
1,신종 코로나바이러스감염증 예방수칙 마스크 착용 흐르는물 30초이상 손씻기 중국방문...,0
2,감염증 대응을 위해 구청장 중심으로 24시간 대책본부를 운영하고 있습니다. 확진자는...,0
3,"중국 방문 후 14일 이내에 발열(37.5도 이상) 또는 호흡기증상(기침,인후통 등...",0
4,"중국 방문 후 14일이내에 발열(37.5도이상) 또는 호흡기증상(기침,인후통등) 발...",0


In [36]:
pip install konlpy

In [37]:
# 불용어 사전 불러오기

# 한글 불용어 사전 파일 경로
stopwords_path = '/content/drive/MyDrive/Colab Notebooks/stopwords-ko.txt'

# 파일 읽기
with open(stopwords_path, "r", encoding="utf-8") as file:
    stopwords = file.readlines()

# 각 불용어의 좌우 공백과 개행문자 제거
stopwords = [word.strip() for word in stopwords]
# 추가 불용어
stopwords.extend(['[', ']'])
print(stopwords)

['!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '...', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '<', '=', '>', '?', '@', '\\', '^', '_', '`', '|', '~', '·', '—', '——', '‘', '’', '“', '”', '…', '、', '。', '〈', '〉', '《', '》', '가', '가까스로', '가령', '각', '각각', '각자', '각종', '갖고말하자면', '같다', '같이', '개의치않고', '거니와', '거바', '거의', '것', '것과 같이', '것들', '게다가', '게우다', '겨우', '견지에서', '결과에 이르다', '결국', '결론을 낼 수 있다', '겸사겸사', '고려하면', '고로', '곧', '공동으로', '과', '과연', '관계가 있다', '관계없이', '관련이 있다', '관하여', '관한', '관해서는', '구', '구체적으로', '구토하다', '그', '그들', '그때', '그래', '그래도', '그래서', '그러나', '그러니', '그러니까', '그러면', '그러므로', '그러한즉', '그런 까닭에', '그런데', '그런즉', '그럼', '그럼에도 불구하고', '그렇게 함으로써', '그렇지', '그렇지 않다면', '그렇지 않으면', '그렇지만', '그렇지않으면', '그리고', '그리하여', '그만이다', '그에 따르는', '그위에', '그저', '그중에서', '그치지 않다', '근거로', '근거하여', '기대여', '기점으로', '기준으로', '기타', '까닭으로', '까악', '까지', '까지 미치다', '까지도', '꽈당', '끙끙', '끼익', '나', '나머지는', '남들', '남짓', '너', '너희', '너희들', '네', '넷', '년', '논하지 않다', '놀라다', '누가 알겠는가', '누구', '다른', '다른 방면으로', '

In [39]:
# 불용어 처리

# Okt 형태소 분석기 생성
okt = Okt()

# 특수 단어 리스트 정의
special_words = ['확진자', '서울의료원']

# 텍스트 전처리 함수 정의
def preprocess_text_korean(text):
    # 특수 문자 제거
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)

    # 형태소 분석 및 불용어 제거
    tokens = okt.pos(text, stem=True)

    # 명사와 동사/형용사만 남기기 및 불용어 제거
    tokens = [word for word, pos in tokens if pos in ['Noun', 'Verb', 'Adjective'] and word not in stopwords]

    # 특정 단어들을 하나의 토큰으로 결합
    for special_word in special_words:
        while special_word in tokens:
            index = tokens.index(special_word)
            tokens[index:index + len(special_word)] = [special_word]

    # 토큰을 다시 문자열로 합침
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# '송출내용' 열에 전처리 적용
df['preprocessed_송출내용'] = df['송출내용'].apply(preprocess_text_korean)

# 변경된 내용을 확인하기 위해 데이터프레임의 처음 몇 행 출력
df[['송출내용', 'preprocessed_송출내용']].head(10)


,송출내용,preprocessed_송출내용
0,[중구청] 오늘 07:00경 중구 장충동 앰버서더호텔 내 화재 발생. 이 지역을 우...,중구청 오늘 경 중구 장충동 앰버 서다 호텔 내 화재 발생 지역 우회 하다 주시 인...
1,[광진구청] 신종 코로나바이러스감염증 예방수칙 마스크 착용 흐르는물 30초이상 손씻...,광진 구청 신종 코로나바이러스 감염증 예방 칙 마스크 착용 흐르다 물 초이 손씻기 ...
2,[송파구청]감염증 대응을 위해 구청장 중심으로 24시간 대책본부를 운영하고 있습니다...,송파 구청 감염증 대응 위해 청장 중심 대책 본부 운영 확 진자 없다 발열 증상 시...
3,"[강남구]중국 방문 후 14일 이내에 발열(37.5도 이상) 또는 호흡기증상(기침,...",강남구 중국 방문 후 이내 발열 호흡기 증상 기침 후통 발 현시 강남구 재난 안전 ...
4,"[강남구]중국 방문 후 14일이내에 발열(37.5도이상) 또는 호흡기증상(기침,인후...",강남구 중국 방문 후 내 발열 호흡기 증상 기침 후통 발현 시 강남구 재난 안전 대...
5,[은평구청]은평성모병원환자코로나19추가확진자발생.성모병원부분일시폐쇄.발열호흡기증상시...,은평 구청 은평 성모 병원 환자 코로나 추가 확 진자 발생 성모 병원 부분 일시 폐...
6,"[강동구]코로나19 2번째 확진자 발생. 성내동 거주 66세(중국인), 2.24. ...",강동구 코로나 확 진자 발생 성내동 거주 중국인 확진 판정 흑석동 중대 병원 격리 ...
7,"[송파구] 코로나19 확진자 2명 추가 발생, 23일 문정동 확진자의 가족으로 서울...",송파구 코로나 확 진자 명 추가 발생 문정동 확 진자 가족 의료 격리 조치 하다 자...
8,"[은평구청]금일코로나19확진추가2명발생,2.1부터은평성모병원입퇴원주민중발열 호흡기증...",은평 구청 금일 코로나 확진 추가 발생 은평 성모 병원 입 퇴원 주민 발열 호흡기 ...
9,"[강동구] 코로나19 확진자 2명 추가 발생, 자세한내용은 강동구 홈페이지를 참고 ...",강동구 코로나 확 진자 명 추가 발생 자세하다 내용 강동구 홈페이지 참고 바라다


In [43]:
# TF-IDF 벡터라이저 생성
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)  # max_features는 필요에 따라 조정 가능

# 텍스트 데이터를 학습 및 변환
X = vectorizer.fit_transform(df['preprocessed_송출내용'])

# 결과 매트릭스의 형태 출력
# print(X.shape)
print(X)

  (0, 1257)	0.05746053921397872
  (0, 2683)	0.1724952456606952
  (0, 1632)	0.18565581044346136
  (0, 2251)	0.16044808089265633
  (0, 3142)	0.1830858919272061
  (0, 2796)	0.20490581921501644
  (0, 3148)	0.2319723784501116
  (0, 3769)	0.09618459211310944
  (0, 2590)	0.24432272176611064
  (0, 3234)	0.1534384459764322
  (0, 1287)	0.11130311593645226
  (0, 3949)	0.21163134481694953
  (0, 3905)	0.321091205839965
  (0, 1781)	0.33980671106406335
  (0, 2275)	0.37629333140506854
  (0, 2952)	0.37629333140506854
  (0, 3176)	0.24220789296210904
  (0, 2457)	0.17395223439855853
  (0, 3177)	0.1925196182501291
  (1, 1722)	0.2364177106828535
  (1, 1428)	0.12540163951637284
  (1, 3211)	0.15899299203986733
  (1, 3908)	0.25305877354553424
  (1, 1289)	0.23127736111527716
  (1, 1307)	0.1494655529532814
  :	:
  (9346, 1257)	0.05365756892001238
  (9346, 3142)	0.17096852898964926
  (9346, 3769)	0.08981870777670842
  (9346, 3234)	0.2865665412327732
  (9347, 1197)	0.3261291464680017
  (9347, 3250)	0.2832267263416